In [23]:
import os
import scipy
import scipy.cluster
import sklearn
import sklearn.cluster
import numpy as np
from PIL import Image
from PIL import ImageColor

In [24]:
def write(json_data: dict, file_name: str):
    with open(file_name, 'w') as f:
        f.write(json.dumps(json_data))

In [25]:
# Download list of all currencies
import urllib
import json

url = 'https://api.coingecko.com/api/v3/coins/list?include_platform=true'
response = urllib.request.urlopen(url)
currencies = json.loads(response.read())
write(currencies, 'tmp_step_01.json')
print('Downloaded list of ', len(currencies))

Downloaded list of  12956


In [26]:
# Include only native or ethereum ERC20s

def include(currency: dict) -> bool:
    if len(currency['platforms']) == 0:
        return True
    if 'ethereum' in currency['platforms']:
        return len(currency['platforms']['ethereum']) != 0
    return False

currencies = list(filter(lambda c: include(c) , currencies))

# Change id to coinGeckoId
for currency in currencies:
    currency['coinGeckoId'] = currency['id']
    currency.pop('id', None )

write(currencies, 'tmp_step_02.json')
print('Filtered out unsupported, remaining count', len(currencies))

Filtered out unsupported, remaining count 6303


In [28]:
# Get image url rank description
import time

# Tmp limit to 100
currency = np.asarray(currencies)[0:100].tolist()
print("Count", len(currency))
i = 0

base_url = 'https://api.coingecko.com/api/v3/coins/'
query_params = '?localization=false&tickers=false&market_data=false&community_data=false&developer_data=false&sparkline=false'
processed_currencies = list()

for currency in currencies:
    url = base_url + currency['coinGeckoId'] + query_params
    response = urllib.request.urlopen(url)
    info = json.loads(response.read())
    currency['description'] = info['description']['en']
    currency['link'] = info['links']['homepage'][0]
    currency['twitter'] = info['links']['twitter_screen_name']
    currency['imageUrl'] = info['image']['large']
    currency['rank'] = info['coingecko_rank']
    write(currencies, 'tmp_step_03.json')
    print(i)
    i += 1
    time.sleep(2)

Count 100
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37


URLError: <urlopen error [Errno 60] Operation timed out>